In [22]:
import calcbench as cb
from calcbench.filing import FilingType
import qgrid

In [3]:
filings = cb.filings_dataframe(company_identifiers=["pep"])

In [55]:
x = None


def filing_timeline(filings):
    global x
    x = filings
    ten_QK = filings[filings.filing_type == FilingType.annualQuarterlyReport]
    ten_KQ_filing_date = pd.NaT
    if ten_QK.shape[0] > 0:
        ten_KQ_filing_date = ten_QK.iloc[0].filing_date
    earnings_press_release = filings[
        filings.filing_type == FilingType.eightk_earningsPressRelease
    ]
    earnings_press_release_filing_date = pd.NaT
    if earnings_press_release.shape[0] > 0:
        earnings_press_release_filing_date = earnings_press_release.iloc[0].filing_date

    return pd.Series(
        (ten_KQ_filing_date, earnings_press_release_filing_date),
        ["10-KQ_filing_date", "earnings_press_release_filing_date"],
    )

In [56]:
filings.groupby(["fiscal_year", "fiscal_period"]).apply(filing_timeline)

10-KQ_filing_date  \
fiscal_year fiscal_period                       
2007.0      1                             NaT   
            2                             NaT   
            3                             NaT   
            4             2008-02-15 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2008.0      1             2008-04-24 00:00:00   
            2             2008-07-23 00:00:00   
            3             2008-10-15 00:00:00   
            4             2009-02-19 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2009.0      1             2009-04-22 00:00:00   
            2             2009-07-22 00:00:00   
            3             2009-10-08 00:00:00   
            4             2010-02-22 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2010.0      1             2010-04-23 00:00:00   
            2             2010-07-20 00:00:00   
            3             2010-10-07 00:00:00   
            4             2011-03-31 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2011.0      1             2011-04-28 00:00:00   
            2             2011-07-21 00:00:00   
            3             2011-10-12 00:00:00   
            4             2012-02-27 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2012.0      1             2012-04-26 00:00:00   
            2             2012-07-25 00:00:00   
            3             2012-10-17 00:00:00   
            4             2013-02-21 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2013.0      1             2013-04-19 00:00:00   
            2             2013-07-24 00:00:00   
            3             2013-10-16 00:00:00   
            4             2014-02-14 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2014.0      1             2014-04-17 00:00:00   
            2             2014-07-23 00:00:00   
            3             2014-10-09 00:00:00   
            4             2015-10-07 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2015.0      1             2015-04-23 00:00:00   
            2             2015-07-09 00:00:00   
            3             2015-10-06 00:00:00   
            4             2016-02-11 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2016.0      1             2016-04-18 16:09:39   
            2             2016-07-07 16:20:31   
            3             2016-09-29 16:20:07   
            4             2017-02-15 16:35:15   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2017.0      1             2017-04-26 00:00:00   
            2             2017-07-11 00:00:00   
            3             2017-10-04 00:00:00   
            4             2018-02-13 00:00:00   
            5                             NaT   
            6                             NaT   
            0                             NaT   
2018.0      1             2018-04-26 00:00:00   
            2             2018-07-10 00:00:00   
            3             2018-10-02 00:00:00   
          

In [12]:
qgrid.show_grid(filings)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [21]:
filings.filing_type

filing_id
2022226              FilingType.eightk_nonfinancial
2008025              FilingType.eightk_nonfinancial
1989389            FilingType.annualQuarterlyReport
1989361      FilingType.eightk_earningsPressRelease
1953295              FilingType.eightk_nonfinancial
1951418              FilingType.eightk_nonfinancial
1951413              FilingType.eightk_nonfinancial
1949352            FilingType.annualQuarterlyReport
1949337      FilingType.eightk_earningsPressRelease
1936451              FilingType.eightk_nonfinancial
1920951              FilingType.eightk_nonfinancial
1909304            FilingType.annualQuarterlyReport
1909244      FilingType.eightk_earningsPressRelease
1904592              FilingType.eightk_nonfinancial
1893878                            FilingType.proxy
1887535              FilingType.eightk_nonfinancial
1871311            FilingType.annualQuarterlyReport
1871280      FilingType.eightk_earningsPressRelease
1819290              FilingType.eightk_nonfinancial
18